# Housing Regression Maching Learning Project

### Imports (Data and Packages)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify, scale_data, ord_encoding, initiate_data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor

%matplotlib inline 

In [2]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)


In [3]:
pickle = pd.read_pickle('lasso_model.pkl')


## Establish Connection with Tableau


## Below is final tableau connection

Before using, you must navigate to the folder with data_preprocessing_final and use the following command: 

set PYTHONPATH=%PYTHONPATH%;folderpath_with_data_preprocessing_final_in_it

Then run the command "tabpy"


In [5]:
housing.columns

Index(['PID', 'SalePrice', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'HeatingQC', 'CentralAir', 'BedroomAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageArea', 'GarageQual', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'Fence', 'MoSold', 'YrSold', 'SaleCondition',
       'TotalBath', 'Bsmt_Unfin_Ratio', 'TotalLivArea'],
      dtype='object')

In [14]:
import tabpy_client
from tabpy.tabpy_tools.client import Client
from data_preprocessing_final import initiate_data

def predict_for_tableau(_arg1, _arg2, _arg3, _arg4,_arg5):
    housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)

    # _arg1 = 'OldTown'
    row = housing[housing['PID'] == 1].copy()
    row['Neighborhood'] = _arg1
    row['OverallQual'] = _arg2
    row['YearBuilt'] = _arg3
    row['LotArea'] = _arg4
    row['BedroomAbvGr'] = _arg5




    # print(row.columns)

    housing.drop(housing[housing['PID'] == 1].index, inplace=True)
    housing = housing.append(row)

    ## Set up train test data
    ## note, random seed is set so that PID == 1 will always be in test daata
    train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)

    ## use the model to predict on the test set and then pull out where PID == 1

    train_data_linear = train_data_linear[train_data_linear['PID'] == 1]
    train_data_linear.drop(['PID'] , axis =1, inplace =True)
    predict = pickle.predict(train_data_linear)
    predict = pd.DataFrame(predict)
    predict.columns = ['Prediction']

    z = np.exp(predict['Prediction'])
    return z.iloc[0]

client = tabpy_client.Client('http://localhost:9004/')
client.deploy('predict_for_tableau', predict_for_tableau, 'predict for tableau', override=True)

In [75]:
final = pd.concat([predict, test_data_linear, test_target_linear], axis =1)
pd.options.display.float_format = '${:,.2f}'.format
final['SalePrice'] = np.exp(final['SalePrice'])
final['Prediction'] = np.exp(final['Prediction'])
final['Pct_diff'] = ((final['Prediction'] - final['SalePrice'])/ final['SalePrice'])*100
final[['Prediction', 'PID','SalePrice', 'Pct_diff']]


,Prediction,PID,SalePrice,Pct_diff
0,"$204,267.84",907181100,"$192,000.00",$6.39
1,"$159,926.91",531452070,"$145,000.00",$10.29
2,"$157,032.98",910204050,"$123,000.00",$27.67
3,"$271,266.09",924151040,"$285,000.00",$-4.82
4,"$150,359.73",923277040,"$138,000.00",$8.96
...,...,...,...,...
477,"$154,205.68",908152110,"$160,500.00",$-3.92
478,"$130,812.19",902427180,"$110,000.00",$18.92
479,"$270,753.79",528180080,"$282,500.00",$-4.16
480,"$136,196.61",534275170,"$151,500.00",$-10.10


## Random Forrest   

In [13]:
# rf_model = RandomForestRegressor(random_state=0)
# # rf_model.fit(train_data_tree, train_target_tree)


RandomForestRegressor(random_state=0)

In [14]:
# train_preds = rf_model.predict(train_data_tree)
# train_mse = mean_squared_error(train_target_tree, train_preds)
# print('Train set MSE is {:.3f}'.format(train_mse))
# print('Train set R2 is {:.3f}'.format(rf_model.score(train_data_tree, train_target_tree)))

# train_preds_test = rf_model.predict(test_data_tree)
# test_mse = mean_squared_error(test_target_tree, train_preds_test)
# print('Test set MSE is {:.3f}'.format(test_mse))
# print('Test set R2 is {:.3f}'.format(rf_model.score(test_data_tree, test_target_tree)))

Train set MSE is 0.002
Train set R2 is 0.987
Test set MSE is 0.014
Test set R2 is 0.900


In [15]:
# predicted = pd.DataFrame(train_preds_test, columns = ['Prediction'])

In [16]:
# final_tree = pd.concat([test_data_tree, predicted, test_target_tree], axis = 1)

In [17]:
# final_tree['SalePrice'] = np.exp(final_tree['SalePrice'])
# final_tree['Prediction'] = np.exp(final_tree['Prediction'])


In [19]:
# ## Look at which features had the most important coefficients 
# sorted(zip(train_data_tree.columns, rf_model.feature_importances_), key = lambda t: t[1], reverse=True)


[('TotalLivArea', 0.611769695940967),
 ('OverallQual', 0.08686860517170654),
 ('YearBuilt', 0.05478411049723338),
 ('ExterQual', 0.049732995132640796),
 ('GarageArea', 0.033503359751477224),
 ('LotArea', 0.01584924789713022),
 ('YearRemodAdd', 0.014954010733209075),
 ('OverallCond', 0.01378381832177892),
 ('GarageYrBlt', 0.009841980604745044),
 ('Fireplaces', 0.00858066722733795),
 ('Bsmt_Unfin_Ratio', 0.008524948559485133),
 ('KitchenQual', 0.007377373224292001),
 ('PID', 0.0066638062850456125),
 ('LotFrontage', 0.005476215459166589),
 ('Neighborhood', 0.004804537389080459),
 ('BsmtFinType1', 0.004671099787307095),
 ('GarageType', 0.0040362385248738355),
 ('MSSubClass', 0.004009441058954551),
 ('OpenPorchSF', 0.0038353831218833796),
 ('CentralAir', 0.0034308679995132658),
 ('MasVnrArea', 0.003281391509908226),
 ('MSZoning', 0.0030934202532820164),
 ('WoodDeckSF', 0.002615876766571731),
 ('MoSold', 0.0026002048637404837),
 ('EnclosedPorch', 0.0024281681202639863),
 ('BsmtQual', 0.00224